In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import joblib

import pandas as pd
import numpy as np

sys.path.append('../')

In [3]:
## Event to text

In [6]:
data_dir = "/Users/andst/synology/phd/datasets/clustering/reproducing/Event"
trained_dir = os.path.join(data_dir, "trained")
os.makedirs(trained_dir, exist_ok=True)

In [78]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

ps = PorterStemmer()

def preprocess(sentence):
    sentence = [word.lower() for word in sentence.split(" ") if word.lower() not in stopwords.words('english')]
    sentence = [ps.stem(word) for word in sentence]
    sentence = [word for word in sentence if word not in stopwords.words('english')]
    
    return sentence

In [79]:
import json



with open(os.path.join(data_dir, "Event_docarr.txt"), "r") as f:
    lines = f.readlines()
    lines = [json.loads(l) for l in lines]


sentences = [line["text"] for line in lines]
sentences = [preprocess(sentence) for sentence in sentences]

In [ ]:
## Train and store event

#dim

In [26]:
a = ' '.join(sentences).split(' ')
df = pd.Series(a).value_counts().reset_index()
df[df[0] > 5]

,index,0
0,in,20020
1,.,15974
2,the,9326
3,attack,8275
4,at,7097
...,...,...
3595,sense,6
3596,priorities,6
3597,rally,6
3598,japan,6


In [89]:
from gensim.models import Word2Vec

model = Word2Vec(sentences=sentences, window=3, vector_size=300, min_count=5, workers=2)

In [32]:
model.wv.save_word2vec_format(os.path.join(trained_dir, "vecs"))

In [88]:
len(model.wv.key_to_index)

3257

In [67]:
model.wv.get_vector()

TypeError: get_vector() missing 1 required positional argument: 'key'

In [48]:
model.wv.get_index("total")

1173

In [ ]:
names = ['Jessa', 'Eric', 'Bob']

with open(r'E:/demos/files_demos/account/sales.txt', 'w') as fp:
    fp.write("\n".join(str(item) for item in names))

In [60]:

# lowercase
# stop words
# porter stemming
# frequencies below 5


In [54]:
"dizingof" in model.wv.key_to_index

False

In [90]:


def save_known_format(model, data_dir: str, dataset: str, lines):
    
    os.makedirs(data_dir, exist_ok=True)
    with open(os.path.join(data_dir, f"{dataset}_word2wid.txt"), "w") as f:
        key_dict = model.wv.key_to_index
        key_dict.pop(".")
        json.dump(model.wv.key_to_index, f)
    word_vec_dict = {key: model.wv.get_vector(key) for key, ind in model.wv.key_to_index.items()}
    joblib.dump(word_vec_dict, os.path.join(data_dir, f"{dataset}_word2vec_300.pkl"))
    
    info_list = []
    for idx, line in enumerate(lines):
        tokens = preprocess(line["text"])
        example = {
            "docid": idx, 
            "topic": line["topic"], 
            "text": line["text"], 
            "tokens": tokens,
            "tokenids": [model.wv.get_index(token) for token in tokens if token in model.wv.key_to_index]
        }
        info_list.append(example)
    return info_list
    with open(os.path.join(data_dir, f"{dataset}_docarr.txt"), "w") as f:
            f.writelines((json.dumps(item) for item in info_list))
    
info_list = save_known_format(model, trained_dir, "Event", lines)

In [ ]:
## Run on DGX

In [91]:
with open(os.path.join(trained_dir, f"Event_docarr.txt"), "w") as f:
    #f.writelines((item for item in info_list))
    json.dump( info_list, f)